In [1]:
from aocd import get_data
from aocd import submit
import unittest

day = 7
year = 2023

def submit_part_a(answer):
    submit(answer, part="a", day=day, year=year)

def submit_part_b(answer):
    submit(answer, part="b", day=day, year=year)

input = get_data(day=day, year=year)

In [2]:
lines = input.split("\n")

In [3]:
from enum import Enum

class HandType(Enum):
    FIVE_OF_A_KIND  = 7
    FOUR_OF_A_KIND  = 6
    FULL_HOUSE      = 5
    THREE_OF_A_KIND = 4
    TWO_PAIRS       = 3
    ONE_PAIR        = 2
    HIGH_CARD       = 1

class Hand:
    card_values_map = {
        '2' : 2,
        '3' : 3,
        '4' : 4,
        '5' : 5,
        '6' : 6,
        '7' : 7,
        '8' : 8,
        '9' : 9,
        'T' : 10,
        'J' : 11,
        'Q' : 12,
        'K' : 13,
        'A' : 14
    }

    card_values_map_with_jocker = {
        'J' : 1,
        '2' : 2,
        '3' : 3,
        '4' : 4,
        '5' : 5,
        '6' : 6,
        '7' : 7,
        '8' : 8,
        '9' : 9,
        'T' : 10,
        'Q' : 12,
        'K' : 13,
        'A' : 14
    }

    def __init__(self, cards, bid):
        self.cards = cards
        self.bid = bid

    def define_nb_cards_from_cards(self, cards):
        unique_cards = set(cards)
        return sorted([cards.count(card) for card in unique_cards], reverse = True)
    
    def define_type_form_nb_cards(self, nb_cards):        
        if nb_cards == [1, 1, 1, 1, 1]:
            return HandType.HIGH_CARD
        if nb_cards == [2, 1, 1, 1]:
            return HandType.ONE_PAIR
        if nb_cards == [3, 1, 1]:
            return HandType.THREE_OF_A_KIND
        if nb_cards == [4, 1]:
            return HandType.FOUR_OF_A_KIND
        if nb_cards == [5]:
            return HandType.FIVE_OF_A_KIND
        if nb_cards == [2, 2, 1]:
            return HandType.TWO_PAIRS
        if nb_cards == [3, 2]:
            return HandType.FULL_HOUSE

    def define_type(self):
        nb_cards = self.define_nb_cards_from_cards(self.cards)
        return self.define_type_form_nb_cards(nb_cards)

    def define_type_with_joker(self):
        nb_cards = self.define_nb_cards_from_cards(self.cards.replace("J", ""))
        if nb_cards == []:
            # case we have 5 joker
            nb_cards = [5]
        else:
            # Joker is here to enforce a strong combination of same card not a weak one
            nb_cards[0] += self.cards.count("J")
        return self.define_type_form_nb_cards(nb_cards)
        
    def comparison_key(self):
        return (self.define_type().value,) + tuple(Hand.card_values_map[card] for card in self.cards)

    def comparison_key_with_joker(self):
        return (self.define_type_with_joker().value,) + tuple(Hand.card_values_map_with_jocker[card] for card in self.cards)

    @staticmethod
    def parse(line):
        cards, bid = line.split()
        return Hand(cards, int(bid))

In [4]:
class HandTest(unittest.TestCase):
    def test_parse_1(self):
        hand = Hand.parse("KKKKK 8765")
        self.assertEqual(hand.cards, "KKKKK")
        self.assertEqual(hand.bid, 8765)

    def test_define_type_1(self):
        hand = Hand.parse("3KTQ9 8765")
        self.assertEqual(hand.define_type(), HandType.HIGH_CARD)

    def test_define_type_2(self):
        hand = Hand.parse("3KTQK 8765")
        self.assertEqual(hand.define_type(), HandType.ONE_PAIR)

    def test_define_type_3(self):
        hand = Hand.parse("KTKQK 8765")
        self.assertEqual(hand.define_type(), HandType.THREE_OF_A_KIND)

    def test_define_type_4(self):
        hand = Hand.parse("KTKKK 8765")
        self.assertEqual(hand.define_type(), HandType.FOUR_OF_A_KIND)

    def test_define_type_5(self):
        hand = Hand.parse("KKKKK 8765")
        self.assertEqual(hand.define_type(), HandType.FIVE_OF_A_KIND)

    def test_define_type_6(self):
        hand = Hand.parse("KQTQK 8765")
        self.assertEqual(hand.define_type(), HandType.TWO_PAIRS)

    def test_define_type_with_joker_1(self):
        hand = Hand.parse("KKJKK 8765")
        self.assertEqual(hand.define_type_with_joker(), HandType.FIVE_OF_A_KIND)

    def test_comparison_key_1(self):
        hand = Hand.parse("KKJKK 8765")
        self.assertEqual(hand.comparison_key(), (HandType.FOUR_OF_A_KIND.value, 13, 13, 11, 13, 13))
        
    def test_comparison_key_with_joker_1(self):
        hand = Hand.parse("KKJKK 8765")
        self.assertEqual(hand.comparison_key_with_joker(), (HandType.FIVE_OF_A_KIND.value, 13, 13, 1, 13, 13))
        
    def test_comparison_key_with_joker_2(self):
        hand = Hand.parse("JJJJJ 8765")
        self.assertEqual(hand.comparison_key_with_joker(), (HandType.FIVE_OF_A_KIND.value, 1, 1, 1, 1, 1))


runner = unittest.TextTestRunner(verbosity=3)
res = runner.run(unittest.TestLoader().loadTestsFromTestCase(HandTest)) 
assert len(res.failures) == 0

test_comparison_key_1 (__main__.HandTest.test_comparison_key_1) ... ok
test_comparison_key_with_joker_1 (__main__.HandTest.test_comparison_key_with_joker_1) ... ok
test_comparison_key_with_joker_2 (__main__.HandTest.test_comparison_key_with_joker_2) ... ok
test_define_type_1 (__main__.HandTest.test_define_type_1) ... ok
test_define_type_2 (__main__.HandTest.test_define_type_2) ... ok
test_define_type_3 (__main__.HandTest.test_define_type_3) ... ok
test_define_type_4 (__main__.HandTest.test_define_type_4) ... ok
test_define_type_5 (__main__.HandTest.test_define_type_5) ... ok
test_define_type_6 (__main__.HandTest.test_define_type_6) ... ok
test_define_type_with_joker_1 (__main__.HandTest.test_define_type_with_joker_1) ... ok
test_parse_1 (__main__.HandTest.test_parse_1) ... ok

----------------------------------------------------------------------
Ran 11 tests in 0.040s

OK


In [5]:
%%time
sorted_hands = sorted((Hand.parse(line) for line in lines), key=lambda hand: hand.comparison_key())
hand_amount_winnings = [hand.bid * (i+1) for i, hand in enumerate(sorted_hands)]

first_answer = sum(hand_amount_winnings)
print(first_answer)

248569531
CPU times: user 14.1 ms, sys: 2.27 ms, total: 16.3 ms
Wall time: 13.8 ms


In [6]:
submit_part_a(first_answer)

aocd will not submit that answer again. At 2023-12-07 07:32:50.852843-05:00 you've previously submitted 248569531 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [7]:
%%time
sorted_hands_with_joker = sorted((Hand.parse(line) for line in lines), key=lambda hand: hand.comparison_key_with_joker())
hand_amount_winnings = [hand.bid * (i+1) for i, hand in enumerate(sorted_hands_with_joker)]

second_answer = sum(hand_amount_winnings)
print(second_answer)

250382098
CPU times: user 10.4 ms, sys: 0 ns, total: 10.4 ms
Wall time: 9.94 ms


In [8]:
submit_part_b(second_answer)

aocd will not submit that answer again. At 2023-12-07 11:37:47.528498-05:00 you've previously submitted 250382098 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 7! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
